In [2]:
import lime_si
import sklearn
import sklearn.datasets
import sklearn.ensemble
import numpy as np
import lime
import lime.lime_tabular
from __future__ import print_function
np.random.seed(1)

Data Import

In [5]:
feature_names = ["Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Marital Status","Occupation", 
                 "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss","Hours per week", "Country"]

data = np.genfromtxt('data/adult.data', delimiter=', ', dtype=str)

Label encoding categorical features and saving encoders in a dictionary

In [6]:
labels = data[:,14]
le= sklearn.preprocessing.LabelEncoder()
le.fit(labels)
labels = le.transform(labels)
class_names = le.classes_
data = data[:,:-1]

categorical_features = [1,3,5, 6,7,8,9,13]

categorical_names = {}
for feature in categorical_features:
    le = sklearn.preprocessing.LabelEncoder()
    le.fit(data[:, feature])
    data[:, feature] = le.transform(data[:, feature])
    categorical_names[feature] = le.classes_
    
    
data = data.astype(float)

np.random.seed(1)
train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(data, labels, train_size=0.80)

/home/kanishka/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


OneHotEncoding for training machine learning model

In [7]:
encoder = sklearn.preprocessing.OneHotEncoder(categorical_features=categorical_features)

scaler = sklearn.preprocessing.StandardScaler(with_mean=False)
scaler.fit(train)

encoder.fit(data)
encoded_train = encoder.transform(train)


import xgboost
gbtree = xgboost.XGBClassifier(n_estimators=300, max_depth=5)
gbtree.fit(encoded_train, labels_train)


predict_fn = lambda x: gbtree.predict_proba(encoder.transform(x)).astype(float)

In [8]:
np.random.seed(1)
i = 407

lime_si.explain_instance_si(training_data=train ,feature_names = feature_names,class_names=class_names, 
                    categorical_features=categorical_features, categorical_names=categorical_names, kernel_width=3,
                    data_row = test[i], predict_fn = predict_fn, num_features=5, label=1)

[('Capital Gain <= 0.00', -0.46625666341571265),
 ('Age <= 28.00', -0.12704138602000872),
 ('Marital Status=Never-married', -0.11980089054753713),
 ('Education-Num <= 9.00', -0.089671786823397423),
 ('Hours per week <= 40.00', -0.087999081370276927)]